In [34]:
import quandl
import pandas as pd
import numpy as np
import datetime
from sklearn.neural_network import MLPRegressor
from sklearn import preprocessing, cross_validation, svm
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

%matplotlib inline

In [20]:
df = quandl.get("WIKI/MSFT")

In [45]:
func = ['mean','std','skew','kurt']
time = [7,30,60,120,252]
k = df.shape[0]-252
Z = pd.DataFrame()
for f in func:
    for t in time:
        aux = pd.DataFrame()
        for i in range(252):
            aux[df.index[i+k]] = list(df.iloc[i+k-t:i+k]['Adj. Close'])
        Z[f+str(t)] = aux.apply(f,axis=0)
Z.tail()

mean7     mean30     mean60    mean120    mean252      std7  \
2018-03-21  94.261429  92.642667  90.988500  85.923496  77.358864  1.275832   
2018-03-22  93.648571  92.681000  91.104833  86.086737  77.472270  0.818320   
2018-03-23  92.988571  92.687000  91.176167  86.222670  77.577813  1.595697   
2018-03-26  92.035714  92.763333  91.205833  86.336687  77.669797  2.643211   
2018-03-27  91.978571  92.952667  91.340333  86.500563  77.788596  2.592974   

               std30     std60    std120    std252     skew7    skew30  \
2018-03-21  2.497495  3.216538  6.258927  9.553977  1.317980 -1.132983   
2018-03-22  2.485449  3.139982  6.171003  9.563211 -0.302638 -1.193683   
2018-03-23  2.477986  3.058279  6.069756  9.551753 -1.512600 -1.200628   
2018-03-26  2.254735  3.009543  5.956163  9.532069 -1.226510 -0.713123   
2018-03-27  2.081467  2.939273  5.887201  9.545600 -1.267540 -0.779877   

              skew60   skew120   skew252     kurt7    kurt30    kurt60  \
2018-03-21 -0.356039 -0.287449  0.412976  2.494047  2.105830 -0.960705   
2018-03-22 -0.402237 -0.289892  0.402984 -1.752289  2.291619 -0.857901   
2018-03-23 -0.401787 -0.294434  0.393050  2.808938  2.351568 -0.775017   
2018-03-26 -0.369305 -0.286647  0.382020  0.792698  0.508059 -0.798161   
2018-03-27 -0.412148 -0.288054  0.374272  0.932427  1.155710 -0.715684   

             kurt120   kurt252  
2018-03-21 -0.816861 -1.133319  
2018-03-22 -0.822501 -1.151439  
2018-03-23 -0.807490 -1.169875  
2018-03-26 -0.793047 -1.182563  
2018-03-27 -0.797423 -1.197794

In [61]:
Z.shape

(252, 20)

In [49]:
df[['Adj. Close']].shift(-1)
W = pd.merge(Z, df[['Adj. Close']].shift(-1),left_index=True,right_index=True)
W = W[:-1]

In [50]:
W.head()

mean7     mean30     mean60    mean120    mean252      std7  \
2017-03-27  63.763902  63.524659  62.687714  60.603842  56.280720  0.279809   
2017-03-28  63.828576  63.573350  62.728526  60.672736  56.331228  0.292053   
2017-03-29  63.887626  63.604796  62.773923  60.744646  56.378025  0.334126   
2017-03-30  63.963547  63.634321  62.834663  60.814793  56.424232  0.393038   
2017-03-31  64.174439  63.673031  62.892167  60.886099  56.470691  0.294412   

               std30     std60    std120    std252     ...        skew30  \
2017-03-27  0.336093  1.060381  2.644926  5.014545     ...     -0.688124   
2017-03-28  0.302928  1.065710  2.626397  5.028673     ...     -0.223858   
2017-03-29  0.323213  1.071880  2.605125  5.047370     ...     -0.130580   
2017-03-30  0.356527  1.060678  2.590000  5.067641     ...      0.099404   
2017-03-31  0.402431  1.065125  2.577213  5.089391     ...      0.388620   

              skew60   skew120   skew252     kurt7    kurt30    kurt60  \
2017-03-27 -0.351811 -0.486165 -0.093777  3.081831  0.403997 -1.382042   
2017-03-28 -0.409672 -0.504710 -0.104637  5.347478 -0.804916 -1.325835   
2017-03-29 -0.459038 -0.517070 -0.114165  4.049286 -0.568833 -1.248117   
2017-03-30 -0.469581 -0.532123 -0.122820  2.591308 -0.319863 -1.190701   
2017-03-31 -0.480649 -0.544022 -0.130517 -0.420144  0.137042 -1.065778   

             kurt120   kurt252  Adj. Close  
2017-03-27 -0.971660 -1.155918   64.255984  
2017-03-28 -0.933185 -1.153629   64.433133  
2017-03-29 -0.900830 -1.154206   64.669332  
2017-03-30 -0.856444 -1.155077   64.816957  
2017-03-31 -0.808319 -1.155463   64.511866  

[5 rows x 21 columns]

In [51]:
X = W.drop(['Adj. Close'],1).fillna(0)
y = W[['Adj. Close']]

In [52]:
mm = MinMaxScaler()
mm.fit(X)
Xs = pd.DataFrame(mm.transform(X))

In [53]:
Xt, Xv, yt, yv = cross_validation.train_test_split(Xs, y, test_size = 0.3)
print Xt.shape

(175, 20)


In [54]:
model = MLPRegressor()

In [55]:
param_grid = dict(
activation = ['identity', 'logistic', 'tanh', 'relu'],
solver = ['lbfgs', 'sgd', 'adam'],
learning_rate = ['constant', 'invscaling', 'adaptive']
)

In [57]:
grid = GridSearchCV(cv = 3, 
                    estimator = model,
                    n_jobs = -1,
                    param_grid=param_grid,
                    verbose = True
)

In [58]:
grid.fit(Xs,y)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


/home/aleespa/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/aleespa/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   34.8s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:  1.4min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'learning_rate': ['constant', 'invscaling', 'adaptive'], 'activation': ['identity', 'logistic', 'tanh', 'relu'], 'solver': ['lbfgs', 'sgd', 'adam']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [59]:
model = grid.best_estimator_
model.fit(Xt,yt)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [60]:
print model.score(Xv,yv)
print model.score(Xt,yt)

0.985289595274
0.986661048288


In [33]:
forecast_prediction = model.predict(mm.transform(X[-1:]))
print(forecast_prediction)

[ 89.40699264]
